# Multilingual Named Entity Recognition

In the series of notebooks till now, we've applied transformers to solve NLP problems on engligh corpora. What if the dataset has multiple languages, mainitainig multiple monolingual models in production will not be any fun.

Fortunatley, we've a class of multilingual transformer to the rescue. Like BERT they are pretrained for masked language modeling as a pretraining objective.

By pretraining on a huge multilingual corpora, we can achieve zero-shot cross lingual transfer. By fine tuning the pretrained label on one language, we can evaluate it on another language without fine tuning for that language explictly.

In this notebook we'll use XLM-RoBERTa pretrained on [2.5Terabyte of text based on Common Crawl Corpus](https://commoncrawl.org/).
The dataset contains only data without parallel texts(translations) and trained an encoder with MLM on this dataset.

Some applications of NER:
* insights from documents
* augmenting quality of search engines
* building a strucutred database from corpus

> Note: *Zero-short transfer or zero-shot learning* usually refers to the task of training a model on one set of labels and then evaluating it on a different set of labels. In the context of transformers, zero-shot learning may also refer to situations where a lnaguage model like GPT-3 is evaluated ona downstream task that it wasn't even fine tuned on.

Problem(assumption):

We want to perfoerm NER for a customer based in Switzerlang, where there are four national languages(With English serving as bridge between them.)

## The Dataset.

We'll use subset of Cross-lingual TRansfer Evaluation of Multilingual Encoder(XTREME) benchmark called WikiANN or PAN-X. This dataset consisits of Wikipedia articles in many languages, including four languages from our Problem. Each article is annotated with `LOC`(location), `PER`(person) and `ORG`(organisation) tags in [IOB2 fornat](https://oreil.ly/yXMUn)

In this format, a `B0` prefix indicates beginning of an entity and consecutive tokens belonging to the same entity are given `I-` prefix. `O` tag indicate that the token does not belong to any entity.

In [1]:
# Sample of the annotation scheme
import pandas as pd
toks = "Jeff Dean is a computer scientist at Google in California".split()
lbls = ["B-PER", "I-PER", "O", "O", "O", "O", "O", "B-ORG", "O", "B-LOC"]
df = pd.DataFrame(data=[toks, lbls], index=['Tokens', 'Tags'])
df

,0,1,2,3,4,5,6,7,8,9
Tokens,Jeff,Dean,is,a,computer,scientist,at,Google,in,California
Tags,B-PER,I-PER,O,O,O,O,O,B-ORG,O,B-LOC


In [2]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
print(f"Number of subsets in XTREME {len(xtreme_subsets)}")
print(f"Subset names in XTREME {xtreme_subsets}")

Number of subsets in XTREME 183
Subset names in XTREME ['XNLI', 'tydiqa', 'SQuAD', 'PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn', 'PAN-X.de', 'PAN-X.el', 'PAN-X.en', 'PAN-X.es', 'PAN-X.et', 'PAN-X.eu', 'PAN-X.fa', 'PAN-X.fi', 'PAN-X.fr', 'PAN-X.he', 'PAN-X.hi', 'PAN-X.hu', 'PAN-X.id', 'PAN-X.it', 'PAN-X.ja', 'PAN-X.jv', 'PAN-X.ka', 'PAN-X.kk', 'PAN-X.ko', 'PAN-X.ml', 'PAN-X.mr', 'PAN-X.ms', 'PAN-X.my', 'PAN-X.nl', 'PAN-X.pt', 'PAN-X.ru', 'PAN-X.sw', 'PAN-X.ta', 'PAN-X.te', 'PAN-X.th', 'PAN-X.tl', 'PAN-X.tr', 'PAN-X.ur', 'PAN-X.vi', 'PAN-X.yo', 'PAN-X.zh', 'MLQA.ar.ar', 'MLQA.ar.de', 'MLQA.ar.vi', 'MLQA.ar.zh', 'MLQA.ar.en', 'MLQA.ar.es', 'MLQA.ar.hi', 'MLQA.de.ar', 'MLQA.de.de', 'MLQA.de.vi', 'MLQA.de.zh', 'MLQA.de.en', 'MLQA.de.es', 'MLQA.de.hi', 'MLQA.vi.ar', 'MLQA.vi.de', 'MLQA.vi.vi', 'MLQA.vi.zh', 'MLQA.vi.en', 'MLQA.vi.es', 'MLQA.vi.hi', 'MLQA.zh.ar', 'MLQA.zh.de', 'MLQA.zh.vi', 'MLQA.zh.zh', 'MLQA.zh.en', 'MLQA.zh.es', 'MLQA.zh.hi', 'MLQA.en.ar', 'MLQA.en.de', 'MLQA.en.vi', 'ML

In [3]:
# We're looking for subsets with PAN
panx_subsets = [subset for subset in xtreme_subsets if subset.startswith("PAN")]
panx_subsets[:3], len(panx_subsets)

(['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg'], 40)

There are 40 PAN-X subsets available.

Eacch of the PAN-X subset is suffixed by language based on [ISO 639-1 language code](https://oreil.ly/R8XNu). This means that to load the German corpus, we pass the `de` code to the `name` argument of `load_dataset()`.

In [4]:
from datasets import load_dataset
# Loading german PAN-X
panx_de_dataset = load_dataset(path="xtreme", name="PAN-X.de")

Found cached dataset xtreme (/Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
panx_de_dataset["train"][0]

{'tokens': ['als', 'Teil', 'der', 'Savoyer', 'Voralpen', 'im', 'Osten', '.'],
 'ner_tags': [0, 0, 0, 5, 6, 0, 0, 0],
 'langs': ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']}

To make a realistic Swiss corpus, we'll same the German(`de`), French(`fr`), Italian(`it`) and English(`en`) corpora from PAN-X according to their spoken proportions. This will create a language imbalance that is very common in real-world datasets, where acquiring labeled examples in aminority language can be expensive due to lack of doman experts who are fluent in the language. This imbalanced dataset will simulate a common situation when working on multilingual applications, we'll cover how we can build a model that works on all languages.

To keep track of each language, let's create a Python `defaultDict` that stores the language code as the key and a PAN-X corpus of type `DatasetDict` as the value:

### Creating multiple languages dataset dict

In [6]:
from collections import defaultdict
from datasets import DatasetDict

langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]

# Return a DatasetDict if a key doesn't exist
panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    # Load monolingual corpus
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    # Shuffle and downsample each split according to fracs
    # Looping through each split
    for split in ds:
        # Assigning sampled split to datasetDict
        panx_ch[lang][split] = ds[split].shuffle(seed=42).select(
            range
            (int(len(ds[split]) * frac))
            )

Found cached dataset xtreme (/Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-50c6130fc2dbe6ad.arrow
Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-3d878c38ca830baa.arrow
Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-c8fed2b7e6d59cbc.arrow
Found cached dataset xtreme (/Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-1d0cd9eb0adc8933.arrow
Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-e0dc2d749c429e9e.arrow
Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-ca0a6f6c69069001.arrow
Found cached dataset xtreme (/Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.it/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.it/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-02142ffc6dacef09.arrow
Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.it/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-79992a3f46d42fd5.arrow
Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.it/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-0e5a91e037304e5e.arrow
Found cached dataset xtreme (/Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-e14b50505509ca06.arrow
Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-529925d4984531e4.arrow
Loading cached shuffled indices for dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-ef60137063549caf.arrow


In [7]:
panx_ch

defaultdict(datasets.dataset_dict.DatasetDict,
            {'de': DatasetDict({
                 train: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 12580
                 })
                 validation: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 6290
                 })
                 test: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 6290
                 })
             }),
             'fr': DatasetDict({
                 train: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 4580
                 })
                 validation: Dataset({
                     features: ['tokens', 'ner_tags', 'langs'],
                     num_rows: 2290
                 })
                 test: Dataset({
                     features: ['tokens', 'ner_tags', 'la

We've used `shuffle()` method to make sure we don't bias our dataset splits accidentally. With `select()` we downsample each corpus according to the values in `fracs`. 

Let's check it out.

In [8]:
import pandas as pd
pd.DataFrame({lang: [panx_ch[lang]["train"].num_rows] for lang in langs},
             index=["Number of training examples"])

,de,fr,it,en
Number of training examples,12580,4580,1680,1180


By design, we have more examples in German than all other languages combined, so we'll use this as a starting point from which we'll perform zero-shot linguagl transfer to french, Italian and English.

In [9]:
# Inspecting a sample from de
element = panx_ch["de"]["train"][0]
for k, v in element.items():
    print(f"{k}: {v}")

tokens: ['Olympique', 'Nîmes', ',', 'Auxerres', 'seinerzeitiger', 'drittklassiger', 'Endspielgegner', ',', 'hatte', 'sich', 'erst', 'gar', 'nicht', 'für', 'die', 'Hauptrunde', 'qualifizieren', 'können', '.']
ner_tags: [3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


> **Note**: In each `Dataset` object, the keys correspond to column names of an Arrow table and values denote the entries in each column.

In [10]:
pd.DataFrame(panx_ch["de"]["train"][0])

,tokens,ner_tags,langs
0,Olympique,3,de
1,Nîmes,4,de
2,",",0,de
3,Auxerres,0,de
4,seinerzeitiger,0,de
5,drittklassiger,0,de
6,Endspielgegner,0,de
7,",",0,de
8,hatte,0,de
9,sich,0,de


In [11]:
# Let's add the ner tag labels to the data
tags = panx_ch["de"]["train"].features["ner_tags"].feature
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [12]:
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}

In [13]:
panx_de = panx_ch["de"].map(create_tag_names)
pd.DataFrame(panx_de["train"][0])

Loading cached processed dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-ea7bb32e9e190d7a.arrow


Loading cached processed dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-f1199d3c03140ace.arrow
Loading cached processed dataset at /Users/jayaprakashsivagami/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-100cbdb51baf3b56.arrow


,tokens,ner_tags,langs,ner_tags_str
0,Olympique,3,de,B-ORG
1,Nîmes,4,de,I-ORG
2,",",0,de,O
3,Auxerres,0,de,O
4,seinerzeitiger,0,de,O
5,drittklassiger,0,de,O
6,Endspielgegner,0,de,O
7,",",0,de,O
8,hatte,0,de,O
9,sich,0,de,O


In [14]:
pd.DataFrame(panx_de["train"][0])[["tokens", "ner_tags_str"]]

,tokens,ner_tags_str
0,Olympique,B-ORG
1,Nîmes,I-ORG
2,",",O
3,Auxerres,O
4,seinerzeitiger,O
5,drittklassiger,O
6,Endspielgegner,O
7,",",O
8,hatte,O
9,sich,O


In [15]:
# Let's check the balance of tags
from collections import Counter

split2freqs = defaultdict(Counter)

for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1

pd.DataFrame.from_dict(split2freqs, orient="index")

,ORG,PER,LOC
train,5397,5881,6169
validation,2639,2870,3172
test,2657,2971,3100


## Multilingual Transformers

Multilingual training and architectures are similar to monolingual counterparts except the corpus consists of documents in multiple languages.

These models might outperform monlingual models and generalize well for a variety of downstream tasks.

To measure the performance of cross-lingual transfer for NER, the [CoNLL-2002](https://huggingface.co/datasets/conll2002) and [CoNLL-2003](https://huggingface.co/datasets/conll2003) datasets are often used as benchmark for English, dutch, spanish and German. This benchmark has all three entity types as PAN-X dataset plus `MISC` for othe entities.

### Multilingual Transformers Evaluation

These models can be evaluated in three different ways:

`en`

Fine-tune on English training data and then evaluate on each langauge set.

`each`

Fine-tune and test on each language individually.

`all`

Fine-tune on all training data to evaluate on all on each langauage's test set.

### Models

mBERT is one of the first multinlingual transformers that differs from BERT only in corpus(multilingual wikipedia articles).

XLM-RoBERTa(or XLM-R for short) has supersed mBERT since then.

XLM-R uses only MLM as a pretraining obejctive for 100 languages. Distinguishing feaure of XLM-R is it uses wikipedia dumps for each languages and 2.5TB of common web crawl data. This is several orders of magnitude larger than corpus used by previous models. This provides good boost for low resource languages where only small wikipedia articles exist.

RoBERTa part of the model name is due to the same pretraining approach used by monlingual RoBERTa models with below significant changes,
* Removes next sentence prediction
* Moves from langauge embeddings to Sentence-Piece tokenizer
* Vocabulary of 25,000 versus 55,000.

XLM-R is a great choice for Multilingual NLU tasks.

### Closer look at Tokenization

Let's compare WordPiece(BERT) tokenizer with SentencePiece(XLM-R) tokenizer.

In [17]:
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"

bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [19]:
# Let's encode a text to find the special tokens used by each
text = "Human's heart is mental!"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

###

In [20]:
pd.DataFrame([bert_tokens, xlmr_tokens])

,0,1,2,3,4,5,6,7,8
0,[CLS],Human,',s,heart,is,mental,!,[SEP]
1,<s>,▁Human,',s,▁heart,▁is,▁mental,!,</s>


BERT uses `[CLS]` and `[SEP]` to indicate start and end of text,
XLM-R uses `<s>` and `</s>` to do the same.

How are these tokens added? Let's see...

### The Tokenizer Pipeline